# 라즈베리파이 실시간 전압/전류 측정

## ⚙ 구성
|항목|모델명|용도|
|-------|-------|-------|
전압 측정|ZMPT101B|220V 전압 감지(출력: 아날로그)
전류 측정|ACS712 (5A/20A/30A)|부하 전류 감지 (출력: 아날로그)
ADC|ADS1115|아날로그 센서 → 디지털 신호로 변환
저장 형식|CSV|전처리 용이, 속도 빠름
측정 주기|100Hz(10ms 간격, 일반 논문에서 자주 사용)

## 🧠 흐름
- ZMPT101B & ACS712 (Analog)
- ADS1115 (I2C → Raspberry Pi)
- 실시간 측정 (0.01초 간격)
- CSV 파일로 저장

## ⚠️ 참고사항
- ACS712 전류 변환 공식: `current (A) = (measured_voltage - Vcc/2) / sensitivity` (예: 20A 모델은 100mV/A, Vcc=5V일 때 2.5V 중심 기준)
- ZMPT101B도 보정이 필요하지만, 일단 상대적 변화 감지용으로도 사용 가능

In [14]:
import time
import board
import busio
import adafruit_ads1x15.ads1115 as ADS
from adafruit_ads1x15.analog_in import AnalogIn
import pandas as pd
import numpy as np

In [15]:
# I2C 및 ADC 설정
i2c = busio.I2C(board.SCL, board.SDA)
ads = ADS.ADS1115(i2c)
ads.gain = 2/3  # ±6.144V

voltage_ch = AnalogIn(ads, ADS.P0)
current_ch = AnalogIn(ads, ADS.P1)

In [16]:
# 센서 보정값
ZMPT_offset = 2.5539
ZMPT_scale = 997.6

ACS712_offset = 2.5087
ACS712_sensitivity = 0.0990

data = []
count = 0

In [17]:
# 초기 측정을 통해 이전값 확보
while True:
    try:
        prev_v_raw = voltage_ch.voltage
        prev_c_raw = current_ch.voltage
        break
    except OSError as e:
        print(f"[초기 I2C 오류] {e}")
        time.sleep(0.001163)

try:
    while True:
        now = time.time()

        try:
            v_raw = voltage_ch.voltage
            c_raw = current_ch.voltage
            prev_v_raw = v_raw
            prev_c_raw = c_raw
        except OSError as e:
            print(f"[I2C 오류 발생] {e}")
            v_raw = prev_v_raw
            c_raw = prev_c_raw

        real_voltage = (v_raw - ZMPT_offset) * ZMPT_scale
        real_current = (c_raw - ACS712_offset) / ACS712_sensitivity

        data.append([now, v_raw, c_raw, real_voltage, real_current, 1])
        count += 1

        if len(data) >= 100:
            df = pd.DataFrame(data, columns=[
                'timestamp', 'v_raw', 'c_raw', 'voltage(V)', 'current(A)', 'label'
            ])

            try:
                df.to_csv("./arc/arc_20sec_60ohm.csv", mode='a', index=False, header=False)
            except Exception as e:
                print(f"[파일 저장 오류] {e}")

            print(f"[{count}] V_raw: {df['v_raw'].mean():.3f}, C_raw: {df['c_raw'].mean():.3f}")
            print(f"      Voltage(V): {df['voltage(V)'].mean():.2f}, Current(A): {df['current(A)'].mean():.2f}")
            print("-" * 60)
            data = []

        time.sleep(0.001163)


except KeyboardInterrupt:
    print("🛑 실시간 감지를 종료합니다.")

[100] V_raw: 2.549, C_raw: 2.521
      Voltage(V): -4.62, Current(A): 0.13
------------------------------------------------------------
[200] V_raw: 2.552, C_raw: 2.520
      Voltage(V): -1.53, Current(A): 0.11
------------------------------------------------------------
[300] V_raw: 2.553, C_raw: 2.517
      Voltage(V): -1.13, Current(A): 0.08
------------------------------------------------------------
[400] V_raw: 2.555, C_raw: 2.522
      Voltage(V): 1.29, Current(A): 0.13
------------------------------------------------------------
[500] V_raw: 2.553, C_raw: 2.513
      Voltage(V): -0.65, Current(A): 0.05
------------------------------------------------------------
[600] V_raw: 2.559, C_raw: 2.523
      Voltage(V): 5.44, Current(A): 0.14
------------------------------------------------------------
[700] V_raw: 2.552, C_raw: 2.523
      Voltage(V): -1.75, Current(A): 0.15
------------------------------------------------------------
[800] V_raw: 2.541, C_raw: 2.521
      Voltage(V):